<a href="https://colab.research.google.com/github/kmouleejunhyuk/bigdata_analysist_practical/blob/main/dacon5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import datetime

raw = pd.read_csv('/content/drive/MyDrive/dacon5/train.csv')
test = pd.read_csv('/content/drive/MyDrive/dacon5/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/dacon5/sample_submission.csv')

#### EDA

In [2]:
raw.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [3]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   object 
 2   본사정원수           1205 non-null   int64  
 3   본사휴가자수          1205 non-null   int64  
 4   본사출장자수          1205 non-null   int64  
 5   본사시간외근무명령서승인건수  1205 non-null   int64  
 6   현본사소속재택근무자수     1205 non-null   float64
 7   조식메뉴            1205 non-null   object 
 8   중식메뉴            1205 non-null   object 
 9   석식메뉴            1205 non-null   object 
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 113.1+ KB


In [4]:
raw.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
본사정원수,1205.0,2807.815768,171.264404,2601.0,2645.0,2760.0,2962.0,3305.0
본사휴가자수,1205.0,157.913693,144.190572,23.0,71.0,105.0,185.0,1224.0
본사출장자수,1205.0,241.142739,43.532298,41.0,217.0,245.0,272.0,378.0
본사시간외근무명령서승인건수,1205.0,274.117012,246.239651,0.0,4.0,299.0,452.0,1044.0
현본사소속재택근무자수,1205.0,43.506224,109.937400,0.0,0.0,0.0,0.0,533.0
중식계,1205.0,890.334440,209.505057,296.0,758.0,879.0,1032.0,1459.0
석식계,1205.0,461.772614,139.179202,0.0,406.0,483.0,545.0,905.0


In [5]:
raw.loc[raw['본사시간외근무명령서승인건수'] == 0, '요일'].value_counts()

수    179
금     34
Name: 요일, dtype: int64

In [6]:
raw.loc[raw['본사시간외근무명령서승인건수'] == 0, '석식계'].mean()

361.84037558685446

#### 데이터 준비

In [9]:
#석식계와 중식계의 데이터타입 변경
raw['중식계'] = raw['중식계'].astype(int)
raw['석식계'] = raw['석식계'].astype(int)
raw['일자'] = pd.to_datetime(raw['일자'])

In [10]:
raw['잔여인원비율'] = (raw['본사정원수'] - raw['본사휴가자수'] - raw['본사출장자수']) / raw['본사정원수']
raw['재택근무활성도'] = raw['현본사소속재택근무자수'] / raw['본사정원수']
raw['시간외근무불가여부'] = [1 if x in ['수', '금'] else 0 for x in raw['요일'] ]
raw['복지휴가제도여부'] = [0 if x < datetime.date(year = 2016, month = 11, day = 1) else 1 for x in raw['일자']]

In [13]:
#중식 데이터와 석식 데이터를 나눕니다(별도 회귀)
lunch = raw.loc[:, raw.columns != '석식메뉴'].copy()
dinner = raw.loc[:, raw.columns != '중식메뉴'].copy()

#### 아웃라이어 제거

In [14]:
#아웃라이어 제거(석식 메뉴가 있음에도 식사 인원이 0인 샘플)
dinner.drop(index = [502, 410], inplace = True)

#석식메뉴가 없는(유효하지 않은) 경우 석식메뉴를 *로 변경
dinner['석식메뉴'] = dinner['석식메뉴'].apply(lambda x: '*' if any(_ in x for _ in ['개발', '계발', '*', '가정', ' ']) else x)

In [15]:
dinner.loc[raw['석식계'] == 0, :]

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,석식메뉴,중식계,석식계,잔여인원비율,재택근무활성도,시간외근무불가여부,복지휴가제도여부
204,2016-11-30,수,2689,68,207,0,0.0,모닝롤/카스텔라 우유/주스 스크램블에그 누룽지탕/쌀밥 (쌀:국내산) 고추장찌개 ...,*,1109,0,0.897731,0.000000,1,1
224,2016-12-28,수,2705,166,225,0,0.0,모닝롤/고구마파이 우유/주스 스크램블에그 누룽지탕/쌀밥 (쌀:국내산) 북어계란국 ...,*,767,0,0.855453,0.000000,1,1
244,2017-01-25,수,2697,79,203,0,0.0,모닝롤/토마토샌드 우유/주스 계란후라이 누룽지탕/쌀밥 (쌀:국내산) 두부젓국찌개 ...,*,720,0,0.895439,0.000000,1,1
262,2017-02-22,수,2632,75,252,0,0.0,모닝롤/브라우니 우유/주스 계란후라이 누룽지탕/쌀밥 (쌀:국내산) 꽃게탕 명엽채...,*,1065,0,0.875760,0.000000,1,1
281,2017-03-22,수,2627,53,235,0,0.0,모닝롤/찐빵 우유/주스 계란후라이 김치죽/쌀밥 (쌀:국내산) 콩나물국 김실파무침...,*,953,0,0.890369,0.000000,1,1
306,2017-04-26,수,2626,45,304,0,0.0,모닝롤/와플 우유/주스 계란후라이 누룽지탕/쌀밥 (쌀:국내산) 된장찌개 검정콩조...,*,835,0,0.867098,0.000000,1,1
327,2017-05-31,수,2637,43,265,0,0.0,모닝롤/베이글 우유/주스 계란후라이 누룽지탕/쌀밥 (쌀:국내산) 두부젓국찌개 자...,*,910,0,0.883201,0.000000,1,1
346,2017-06-28,수,2648,58,259,0,0.0,모닝롤/단팥빵 우유/주스 계란후라이 누룽지탕/쌀밥 (쌀:국내산) 쇠고기매운국 고...,*,745,0,0.880287,0.000000,1,1
366,2017-07-26,수,2839,254,246,0,0.0,모닝롤/단팥빵 우유/주스 계란후라이 누룽지탕/쌀밥 (쌀:국내산) 민물새우찌개 숙...,*,797,0,0.823882,0.000000,1,1
392,2017-09-01,금,2642,177,303,45,0.0,모닝롤/단팥빵 우유/주스 계란후라이 단호박죽/쌀밥 (쌀:국내산) 대구찌개 깻순나...,*,663,0,0.818319,0.000000,1,1


In [16]:
raw['중식메뉴'].values[0].split(sep = ' ')

['쌀밥/잡곡밥',
 '(쌀,현미흑미:국내산)',
 '오징어찌개',
 '',
 '쇠불고기',
 '(쇠고기:호주산)',
 '계란찜',
 '',
 '청포묵무침',
 '',
 '요구르트',
 '',
 '포기김치',
 '(배추,고추가루:국내산)',
 '']

In [17]:
raw['중식메뉴'].values[100].split(sep = ' ')

['쌀밥/잡곡밥',
 '(쌀:국내산)',
 '버섯들깨찌개',
 '',
 '닭오븐구이',
 '',
 '김치전',
 '',
 '치커리사과무침',
 '',
 '열무김치',
 '(김치:국내산)',
 '',
 '',
 '']